## Create monthly boundary condition forcing files based on Temperature and Salinity

In [1]:
import netCDF4 as nc
import numpy as np
import pandas as pd
import matplotlib
import xarray as xr

%matplotlib inline

#### Parameters

In [2]:
# domain dimensions:
imin, imax = 1139, 2179
jmin, jmax = 159, 799

# Rimwidths:
rimwidthN  = 10
rimwidthS  = 10
rimwidthW  = 10
rimwidthE  = 20

# Boundary coordinates: (i1,i2,j1,j2)
bdy_NCB = (1598,2177,778,798) # Northern Canada Basin 
bdy_WCB = (2168,2178,390,797) # Western Canada Basin 
bdy_LS  = (1140,1150,446,672) # Baffin Bay
bdy_HB  = (1190,1472,291,301) # Hudson Bay

In [3]:
print('Python boundary dimensions (should be same length):')
print('nCB dimensions: ', bdy_NCB[1]-bdy_NCB[0],  bdy_NCB[3]-bdy_NCB[2])
print('wCB dimensions: ', bdy_WCB[1]-bdy_WCB[0],  bdy_WCB[3]-bdy_WCB[2])
print('LS dimensions: ', bdy_LS[1]-bdy_LS[0],  bdy_LS[3]-bdy_LS[2])
print('HB dimensions: ', bdy_HB[1]-bdy_HB[0],  bdy_HB[3]-bdy_HB[2]) # index in 0:5 should be one longer

Python boundary dimensions (should be same length):
nCB dimensions:  579 20
wCB dimensions:  10 407
LS dimensions:  10 226
HB dimensions:  282 10


In [4]:
print('Fortran boundary dimensions')
print('nCB dimensions: ', 1038-460) # 638
print('wCB dimensions: ', 638-232) # 1038
print('LS dimensions: ', 513-288) # 2
print('HB dimensions: ', 333-52) # 133

Fortran boundary dimensions
nCB dimensions:  578
wCB dimensions:  406
LS dimensions:  225
HB dimensions:  281


#### Files

In [5]:
mesh         = nc.Dataset('/ocean/brogalla/GEOTRACES/data/ANHA12/ANHA12_mask_Pb-20220317.nc')
mesh_lon     = np.array(mesh.variables['nav_lon'])
mesh_lat     = np.array(mesh.variables['nav_lat'])
mesh_bathy   = np.array(mesh.variables['tmask'][0,:,:,:])
bathy_masked = np.ma.masked_where((mesh_bathy> 0.1), mesh_bathy)
ocean_masked = np.ma.masked_where((mesh_bathy<0.1), mesh_bathy)
depths       = np.array(mesh.variables['nav_lev'])

#### Functions

In [16]:
def load_climatology(year, month):        
    # load year 2015 climatology of ANHA12 gridT file:
    df_gridT = xr.open_dataset(f'/data/brogalla/ANHA12/new_averages/gridT_{year}_{month}.nc')
    
    bdy_temp = df_gridT['votemper'].values[:,:,:]
    bdy_sal  = df_gridT['vosaline'].values[:,:,:]
    
    return bdy_temp, bdy_sal

In [7]:
def create_LS_BC(bdy_temp, bdy_sal, bdy_depth, mask=False):
    # Pb end-members:
    BBDW_Pb   = 2e-12  # Baffin Bay deep water Pb
    AO_Pb     = 6e-12  # Arctic water Pb
    AW_Pb     = 30e-12 # Atlantic water Pb
    AOAW_Pb   = 25e-12 # Arctic-outflow Atlantic water Pb
    AW_temp   = 5.2    # Atlantic Water temperature
    BBDW_temp = 2.5    # Baffin Bay deep water temperature 
    AW_sal    = 35.2   # Atlantic Water salinity
    PML_hot_Pb = 30e-12 # summer polar mixed layer Pb

    # Linear interpolate salinity endpoints:
    end_sal = [34.0, AW_sal]
    end_dPb = [AO_Pb, AOAW_Pb]
    dPb_BC  = np.interp(bdy_sal, end_sal, end_dPb)
        
    # Baffin Bay deep water:
    end_temp_BBDW = [BBDW_temp, AW_temp]
    end_dPb_BBDW  = [BBDW_Pb, AOAW_Pb]
    dPb_BC[(bdy_sal > 34.75) & (bdy_temp <= 4.9)] = np.interp(bdy_temp[(bdy_sal > 34.75) & (bdy_temp <= 4.9)], \
                                                                  end_temp_BBDW, end_dPb_BBDW)    
 
    # Polar Mixed Layer:
    if mask: # mask summer solar heating influence:
        for row in range(0,10):
            for loc in range(0,225):
                for dep in range(0,50):
                    if (dep < 24) & (dPb_BC[dep,row,loc] > 15e-12):
                        if np.nanmean(dPb_BC[dep:56-dep,row,loc]) < dPb_BC[dep,row,loc]:
                            dPb_BC[dep,row,loc] = np.nanmean(dPb_BC[dep:56-dep,row,loc])
    
    # Atlantic Water off Greenland:
    grid = 150 # Grid point of central BB
    boundary_sal = 35.0 # salinity of outer-edge of Atlantic Water core off Greenland
    end_sal_AW   = [boundary_sal, AW_sal]
    end_dPb_AW   = [AOAW_Pb, AW_Pb]
    dPb_BC[:,:,grid:][(bdy_sal[:,:,grid:] > boundary_sal) & (bdy_temp[:,:,grid:] > AW_temp)] =  \
        np.interp(bdy_sal[:,:,grid:][(bdy_sal[:,:,grid:] > boundary_sal) & (bdy_temp[:,:,grid:] > AW_temp)], end_sal_AW, end_dPb_AW)

    return dPb_BC

In [8]:
def create_AO_BC(bdy_temp, bdy_sal):
    # Pb end-members:
    PML_Pb   = 3e-12   # Polar Mixed Layer Pb
    ACW_Pb   = 8e-12  # Alaskan Coastal Water Pb
    wBSW_Pb  = 3e-12   # Winter Bering Sea Water Pb
    AL_Pb    = 5e-12   # Atlantic Water Pb
    CBDW_Pb  = 2e-12   # Canada Basin deep water Pb
    
    PML_sal   = 29      # Polar mixed layer salinity max
    PML_temp  = 0       # Polar mixed layer temperature
    ACW_sal   = 32.5    # ACW salinity
    ACW_temp  = 2.5       # ACW temperature
    wBSW_temp = 34.8    # Winter Bering Sea Water temperature
    wBSW_sal  = 32.5    # Winter Bering Sea Water salinity
    AL_temp   = 0.1     # Atlantic Water temperature min
    AL_sal    = 34.8    # Atlantic Water salinity min
    CBDW_temp = -0.5    # Canada Basin deep water temperature
    CBDW_sal  = 35.0    # Canada Basin deep water salinity
    
    # Linear interpolate endpoints between PML and ACW:
    end_temp = [PML_temp, ACW_temp]
    end_dPb = [PML_Pb, ACW_Pb]
    dPb_BC  = np.interp(bdy_temp, end_temp, end_dPb)

    # wBSW to AL
    end_sal_AL = [wBSW_sal, AL_sal]
    end_dPb_AL = [wBSW_Pb, AL_Pb]
    dPb_BC[(bdy_sal > wBSW_sal) & (bdy_sal < AL_sal)] = np.interp(bdy_sal[(bdy_sal > wBSW_sal) & (bdy_sal < AL_sal)], \
                                                                  end_sal_AL, end_dPb_AL)
    
    # AL to CBDW
    end_temp_CBDW = [CBDW_temp, AL_temp]
    end_dPb_CBDW  = [CBDW_Pb, AL_Pb]
    dPb_BC[(bdy_sal > AL_sal)] = np.interp(bdy_temp[(bdy_sal > AL_sal)], end_temp_CBDW, end_dPb_CBDW)
    
    
    return dPb_BC

In [9]:
def create_HB_BC(bdy_temp, bdy_sal):
    # Pb end-members:
    PML_Pb   = 3e-12   # Polar Mixed Layer Pb
    AL_Pb    = 5e-12   # Atlantic Water Pb
    
    PML_sal   = 29      # Polar mixed layer salinity max
    AL_sal    = 34.8    # Atlantic Water salinity min
    
    # Linear interpolate endpoints between PML and ACW:
    end_sal = [PML_sal, AL_sal]
    end_dPb = [PML_Pb, AL_Pb]
    dPb_BC  = np.interp(bdy_sal, end_sal, end_dPb)

    return dPb_BC

In [10]:
def flatten_input(var, order):
    b = var[0,:,:].flatten(order=order)
    for i in range(1,len(var)):
        a = var[i,:,:].flatten(order=order)
        b = np.vstack((b,a))
    return b

In [11]:
def reshape_boundary(rimwidth, boundary_dPb, order):
    
    dPb_O = flatten_input(boundary_dPb, order)
    
    dPb_OBC = np.reshape(dPb_O, (1,50,1,np.max(boundary_dPb.shape)*rimwidth))

    return dPb_OBC

In [20]:
def save_file(year, month):    
    file_write = xr.Dataset(
        {'dPb_N' : (("time_counter","deptht","y","x1"), dPb_north_BC), \
         'dPb_S' : (("time_counter","deptht","y","x2"), dPb_south_BC), \
         'dPb_E' : (("time_counter","deptht","y","x3"), dPb_east_BC), \
         'dPb_W' : (("time_counter","deptht","y","x4"), dPb_west_BC)}, 
        coords = {
            "time_counter": np.zeros(1),
            "deptht": np.zeros(50),
            "y": np.zeros(1),
            "x1": np.zeros(np.max(dPb_north_BC.shape)),
            "x2": np.zeros(np.max(dPb_south_BC.shape)),
            "x3": np.zeros(np.max(dPb_east_BC.shape)),
            "x4": np.zeros(np.max(dPb_west_BC.shape))
        },
    )
    file_write.to_netcdf(f'/ocean/brogalla/GEOTRACES/data/Pb-202209-forcing/Pb_OBC_y{year}m{month:02}.nc', \
                         unlimited_dims='time_counter')
    
    return

#### Calculate

In [33]:
# Load climatology conditions: 
year = 2005

LS_T  = np.zeros((12,50,bdy_LS[1]-bdy_LS[0],bdy_LS[3]-bdy_LS[2]));     LS_S  = np.zeros((12,50,bdy_LS[1]-bdy_LS[0],bdy_LS[3]-bdy_LS[2]));
nCB_T = np.zeros((12,50,bdy_NCB[1]-bdy_NCB[0],bdy_NCB[3]-bdy_NCB[2])); nCB_S = np.zeros((12,50,bdy_NCB[1]-bdy_NCB[0],bdy_NCB[3]-bdy_NCB[2]));
wCB_T = np.zeros((12,50,bdy_WCB[1]-bdy_WCB[0],bdy_WCB[3]-bdy_WCB[2])); wCB_S = np.zeros((12,50,bdy_WCB[1]-bdy_WCB[0],bdy_WCB[3]-bdy_WCB[2]));
HB_T  = np.zeros((12,50,bdy_HB[1]-bdy_HB[0],bdy_HB[3]-bdy_HB[2]));     HB_S  = np.zeros((12,50,bdy_HB[1]-bdy_HB[0],bdy_HB[3]-bdy_HB[2]));

for month in range(1,13):
    temp, sal  = load_climatology(year, f'{month:02}')
    
    LS_T[month-1,:,:,:]  = temp[:,bdy_LS[0]:bdy_LS[1],bdy_LS[2]:bdy_LS[3]]
    LS_S[month-1,:,:,:]  = sal[:,bdy_LS[0]:bdy_LS[1],bdy_LS[2]:bdy_LS[3]]
    nCB_T[month-1,:,:,:] = temp[:,bdy_NCB[0]:bdy_NCB[1],bdy_NCB[2]:bdy_NCB[3]]
    nCB_S[month-1,:,:,:] = sal[:,bdy_NCB[0]:bdy_NCB[1],bdy_NCB[2]:bdy_NCB[3]]
    wCB_T[month-1,:,:,:] = temp[:,bdy_WCB[0]:bdy_WCB[1],bdy_WCB[2]:bdy_WCB[3]]
    wCB_S[month-1,:,:,:] = sal[:,bdy_WCB[0]:bdy_WCB[1],bdy_WCB[2]:bdy_WCB[3]]
    HB_T[month-1,:,:,:]  = temp[:,bdy_HB[0]:bdy_HB[1],bdy_HB[2]:bdy_HB[3]] 
    HB_S[month-1,:,:,:]  = sal[:,bdy_HB[0]:bdy_HB[1],bdy_HB[2]:bdy_HB[3]]

In [34]:
# Create boundary conditions:
LS_depth = np.tile(depths, (bdy_LS[3]-bdy_LS[2],rimwidthS,1)).transpose()
dPb_LS   = np.zeros((12,50,bdy_LS[1]-bdy_LS[0],bdy_LS[3]-bdy_LS[2]))
dPb_nCB  = np.zeros((12,50,bdy_NCB[1]-bdy_NCB[0],bdy_NCB[3]-bdy_NCB[2]))
dPb_wCB  = np.zeros((12,50,bdy_WCB[1]-bdy_WCB[0],bdy_WCB[3]-bdy_WCB[2]))
dPb_HB   = np.zeros((12,50,bdy_HB[1]-bdy_HB[0],bdy_HB[3]-bdy_HB[2]))

for month in range(1,13):
    if (month > 5) & (month < 12):
        dPb_LS[month-1,:,:,:]  = create_LS_BC(LS_T[month-1] , LS_S[month-1], LS_depth, mask=True)
    else:
        dPb_LS[month-1,:,:,:]  = create_LS_BC(LS_T[month-1] , LS_S[month-1], LS_depth, mask=False)
    dPb_nCB[month-1,:,:,:] = create_AO_BC(nCB_T[month-1], nCB_S[month-1])
    dPb_wCB[month-1,:,:,:] = create_AO_BC(wCB_T[month-1], wCB_S[month-1])
    dPb_HB[month-1,:,:,:]  = create_HB_BC(HB_T[month-1] , HB_S[month-1])
    
dPb_LS[np.isnan(dPb_LS)]   = 3e-12
dPb_nCB[np.isnan(dPb_nCB)] = 3e-12
dPb_wCB[np.isnan(dPb_wCB)] = 3e-12
dPb_HB[np.isnan(dPb_HB)]   = 3e-12

## or base on bathy

In [35]:
dPb_wCBr = np.flip(dPb_wCB, axis=2)
dPb_nCBr = np.flip(dPb_nCB, axis=3)

#### Reshape boundary conditions and write to file

In [36]:
for month in range(1,13):
    dPb_north_BC = reshape_boundary(rimwidthN, dPb_wCBr[month-1], 'C')
    dPb_east_BC  = reshape_boundary(rimwidthE, dPb_nCBr[month-1], 'F')
    dPb_south_BC = reshape_boundary(rimwidthS, dPb_LS[month-1], 'C')
    dPb_west_BC  = reshape_boundary(rimwidthW, dPb_HB[month-1], 'F')
    
    save_file(year, month)

In [37]:
# Check that there are no NaN values in the files:
print('North BC # of NaNs:', sum(dPb_north_BC[np.isnan(dPb_north_BC)]))
print('East BC # of NaNs: ', sum(dPb_east_BC[np.isnan(dPb_east_BC)]))
print('South BC # of NaNs:', sum(dPb_south_BC[np.isnan(dPb_south_BC)]))
print('West BC # of NaNs: ', sum(dPb_west_BC[np.isnan(dPb_west_BC)]))

North BC # of NaNs: 0
East BC # of NaNs:  0
South BC # of NaNs: 0
West BC # of NaNs:  0
